In [22]:
import pymongo
import datetime
from splinter import Browser
from bs4 import BeautifulSoup as bs
from pprint import pprint
import pandas as pd
import requests

In [25]:
!which chromedriver

/c/Users/court/anaconda3/envs/PythonData/Library/mingw-w64/bin/chromedriver


In [28]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [29]:
#initialize PyMongo to work with MongoDBs
#conn='mongodb://localhost:27017'
#client = pymongo.MongoClient(conn)

In [30]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [32]:
html=browser.html
new_soup = bs(html, 'html.parser')
slide_elem = new_soup.select_one('ul.item_list li.slide')


In [33]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8812/5-hidden-gems-are-riding-aboard-nasas-perseverance-rover/" target="_self">5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover</a></div>

In [34]:
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover"

In [36]:
news_p = slide_elem.find("div", class_='article_teaser_body').get_text()
news_p

"The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative."

In [37]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [38]:
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [39]:
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [40]:
html=browser.html
image_soup= bs(html, 'html.parser')

In [41]:
image_url_rel=image_soup.select_one('figure.lede a img').get("src")
image_url_rel

'/spaceimages/images/largesize/PIA18049_hires.jpg'

In [42]:
img_url = f'https://www.jpl.nasa.gov{image_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18049_hires.jpg'

In [43]:
df= pd.read_html('https://space-facts.com/mars/')[0]
df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [44]:
df.columns=['Description', 'Mars']
df.set_index('Description', inplace=True)
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [45]:
mars_facts = df.to_html()
mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

In [46]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [47]:
hemisphere_image_urls = []
links = browser.find_by_css("a.product-item h3")
for i in range(len(links)):
    hemisphere={}
    browser.find_by_css("a.product-item h3")[i].click()
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    hemisphere['title'] = browser.find_by_css("h2.title").text
    hemisphere_image_urls.append(hemisphere)
    browser.back()

In [48]:
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [49]:
browser.quit()